# Робо-журналистика. Мини-проект

## Программа для подбора изображений к тексту

Пользователь загружает англоязычный текст и получает файл с ключевыми словами и ссылками на 30 (макисмум) изображений к тексту с сервиса Flickr, которые можно использовать в коммерческих целях без указания авторства (No known copyright restrictions, Public Domain Dedication (CC0), Public Domain Mark). В файле содержится также название изображения и 2 ссылки на него: большой и средний размер.

### Алгоритм

1. Программа принимает на вход файл с англоязычным текстом в формате txt
2. Обработка текста
    + Все слова с маленькой буквы, без знаков препинания
    + Подсчитывается частота каждого слова (Counter)
    + Поиск стоп-слов в тексте (файл "stop-list.txt") и присвоение стоп-словам частоты равной 0
    + Выделение 5 ключевых слов текста
3. Запрос к API Flickr (задается список тегов, поиск изображений со свободной лицензией)
4. Выбор необходимых данных: title, farm, id, server, secret
5. Формирование ссылок на фото
6. Запись необходимых данных в файл "links_to_images.txt": ключевые слова, порядковый номер, название фото, ссылки

<b><i>Андрей Нестеров, МДЖ172</i></b>

In [ ]:
import requests
from collections import Counter
from bs4 import BeautifulSoup
from IPython.display import Image, display

In [ ]:
def get_photo_for_article(text):
    """
    Функция принимает на вход файл с текстом и возвращает файл "links_to_images.txt"
    с ключевыми словами текста и ссылками на 30 (максимум) изображений из сервиса Flickr к тексту.
    Изображения не защищены копирайтом и могут использоваться без указания авторства в коммерческих целях.
    """
    
    list_of_words = [] # Список для всех слов из загруженного файла
    list_of_stop_words = [] # Список для слов из стоп-листа
    list_of_keywords = [] # Список для ключевых слов
    dict_of_urls = {} # Словарь для хранения ссылок в виде {'id': ('название фото', 'ссылки')}
    list_of_data = [] # Список для хранения строк, который будет записан в файл links_to_images.txt
    preview_links = [] # Список для ссылок на превью 
    
    with open(text) as file:
        original = file.read()
        
    for lines in original.strip().split(): # Делим весь текст на слова и добавляем их в список
        lines_no_punct = lines.lower().rstrip(".").rstrip(",").rstrip(")").lstrip("(") # С маленькой буквы без знаков препинания
        list_of_words.append(lines_no_punct)

    with open("stop-list.txt") as stop_list: # Открываем файл "стоп-лист"
        stop_list_original = stop_list.read()

    for elements in stop_list_original.split(): # Добавляем все слова из стоп-листа в список
        list_of_stop_words.append(elements)
    
    cntr = Counter(list_of_words) # Частота слова в тексте 
    statistic = dict(cntr) # Превращаем counter в словарь

    for words in list_of_stop_words: # Ищем стоп-слова в тексте
        if words in statistic: # Если слово встречается, то
            statistic[words] = 0 # Приравниваем его частоту к 0

    keys = Counter(statistic).most_common(5) # Выводим топ-5 самых частых слов (ключевые слова)

    for element in keys: # Добавляем ключевые слова в список 
        list_of_keywords.append(element[0])
        
    # Превращаем список в строку формата "keyword, keyword"
    
    keywords = f"{list_of_keywords[0]}, {list_of_keywords[1]}, {list_of_keywords[2]}, {list_of_keywords[3]}, {list_of_keywords[4]}"
    
    # Получаем нужные фото по тегам, запрос в формате XML
    url = "https://api.flickr.com/services/rest/?"
    query = {'api_key':'c949a8a9e9e36da8cfceb75a95ff7980',
             'method': 'flickr.photos.search',
             'content_type':'1', # Ищем только фото
             'tags': keywords, # Задаем список тэгов (ключевых слов)
             'tag_mode': 'any', # Любое соответсвие тэгу
             'per_page':'30', # Максимум 30 фото
             'page':'1', # На первой странице
             'sort':'interestingness-desc', # Сортирвка - по "интересности"
             'license':'7,9,10' # Фото без копирайта
            }

    f = requests.get(url, params=query)
    page = BeautifulSoup(f.text, "lxml")
    for photo in page.html.body.photos('photo'):
        photo_id = f"{photo['id']}"
        photo_title = f"{photo['title']}"
        url_string_large = f"Large - https://farm{photo['farm']}.staticflickr.com/{photo['server']}/{photo['id']}_{photo['secret']}_b.jpg"
        url_string_medium = f"Medium - https://farm{photo['farm']}.staticflickr.com/{photo['server']}/{photo['id']}_{photo['secret']}_z.jpg"
        dict_of_urls[photo_id] = photo_title, url_string_large, url_string_medium
        url_string_preview = f"https://farm{photo['farm']}.staticflickr.com/{photo['server']}/{photo['id']}_{photo['secret']}_m.jpg"
        preview_links.append(url_string_preview)

    for key, value in dict_of_urls.items():
        string_data = f"'{dict_of_urls[key][0]}', {dict_of_urls[key][1]}, {dict_of_urls[key][2]}"
        list_of_data.append(string_data)

    links = open("links_to_images.txt", "w") # Подготавливаем файл, который содержит
    print(f"Keywords: {keywords}", file=links) # Ключевые слова
    for i, elements in enumerate(list_of_data): 
        index = i + 1
        print(index, elements, file=links) # Порядковый номер, название фото, ссылки
    links.close()
    
    print("Your images are ready! Have a look at some of them.\nPlease, open 'links_to_images.txt' in your directory to get all the links.")
    
    for link in preview_links[0:5]: # Показываем 5 превью
        display(Image(link))
        
    return
    #return ("Ready! Open file 'links_to_images.txt' in your directory")

In [ ]:
get_photo_for_article("journ.txt")